# Configuring LDAP and Authenticator in Tapis v3

## Introduction

This document describes the options available for configuring LDAP connections for a v3 tenant, and it includes
examples of the 4 primary LDAP connections used commonly in the TACC-hosted Tapis instance.

The four examples are:

1. All accounts in the TACC LDAP.
2. All accounts in the "v3 dev" LDAP.
3. A subset of accounts in the TACC LDAP where the subset is defined by a host filter; e.g., (host=3dem.org)
4. A subset of accounts in the v2 "agaveldap" where the subset is defined by a specific OU; e.g., ou=tenantirec
  
In providing definitions of each of the 4 examples above, we will discuss the different configuration options 
available, which include:

1. Configurations on the `/v3/tenants/ldaps` object, including the `user_dn` attribute.
2. Configurations on the Authenticator's "ldapObject" type custom_idp_configuration for the tenant, including the
     attributes `user_search_prefix`, `user_search_supplemental_filter`, and `user_search_filter` and how the 
     "ldapObject" configuration, when provided, relates to the `/v3/tenants/ldaps` object configuration.

We'll aslo discuss how these relate to the LDAP bind credential, and the cases that necessitate new objects being
created for a tenant.

## Custom IdP Configuration vs Tenant LDAP Object Definitions

At a high level, there are two sources of configuration for an LDAP IdP in v3: the Tenant LDAP object, which is 
required, and the "ldapObject" custom IdP confguration within the authenticator service for the tenant, which is 
optional. We explain both here.


### Tenant LDAP Object Definitions
Every Tapis tenant using the Authenticator to define an IdP using an LDAP server must make use of an LDAP object definition defined in the Tenants service and managed under the `/v3/tenants/ldaps` endpoint. Each such LDAP object includes: 

1. An LDAP server configuration 
2. The LDAP `user_dn`, which at a minimum specifies the OU within the LDAP server to use, and 
3. The `bind_dn` and the `bind_credential` which are used to connect to the LDAP server, list accounts, and    validate passwords. 

There are a number of such LDAP objects definied already in the TACC-hosted Tapis v3 instance, incuding: 

1. `tacc-all` -- All accounts in the TACC LDAP 
2. `tapis-dev`-- A specific OU within the "Dev LDAP", a developent LDAP server that runs within the Tapis k8s cluster at TACC. This LDAP server can support multiple tenants with different OUs. The `tapis-dev` Tenant LDAP object points to the `ou=tenant.dev` organizational unit which has accounts such as "testuser1", "testuser2", etc.
3. `tapis-training` -- A different OU within the "Dev LDAP" server referenced in 2). This OU is used for Tapis trainings; specfically, it points to `ou=tenant.training`.
4. `ldap-vdj` -- Points to the `ou=tenantvdjserver-org` OU within the agaveldap server (also referred to as the "Tapis v2 LDAP server") 
5. `uh-ci` -- The primary LDAP server for users at the University of Hawaii.

Note that a single bind credential can be used across multple LDAP objects -- for example, the same bind credential
(`ldap.tapis-dev`) is used for both `tapis-dev` and `tapis-training`. A large number of Tapis tenants can make use
of an existing Tenant LDAP object -- for example, any tenant wanting to use "all TACC accounts" can use the `tacc-all` tenant object. However, if the LDAP server or the OU defined in the Tenant LDAP object do not match the needs
of the tenant, a new object must be created. For example, tenants using the v2 "agaveldap" server will need their
own Tenant LDAP object which points to the specific OU for their tenant. In this case, the bind credential can still
be reused. 


### Custom IdP Configurations within Authenticator
The authenticator service maintains custom confgurations in its SQL database for each tenant it serves, and one of the (optional) custom configurations available to a tenant is the "custom_idp_configuration". Currently, a tenant can have either 0 or 1 custom_idp_configurations, which are JSON objects that conform to the JSONSchema defined in the authenticator repository here: 
    https://github.com/tapis-project/authenticator/blob/prod/service/resources/custom-idp-configschema.json

The different types of configurations available are specified as object type definitions in the schema and include definitions such as "githubObject" (for using GitHub as the IdP), "taccKeycloakObject" (for using the TACC KeyCloak instance as the IdP, including support for Globus Auth), and "ldapObject", for adding additional configurations to an LDAP IdP. This last one is the one we are focusing on here. **Note that the use of an "ldapObject" still requires a Tenant LDAP Object to be defined, as described in the previous section above.** But, it can supplement the definition with additional configuration, including:

1. Default page limit size -- to change the number of profiles that are returned by default.
2. Attributes that modify the user search filter, to further restrict which LDAP users within an OU can authenticate to the tenant. **Note: the user search filter is orthogonal to the user DN defined in the Tenant LDAP object.**
  

#### "ldapObject" User Search Filter Attribute Details
In LDAP, the user search filter is a separate attribute from the user DN which is used to filter the user objects
allowed to bind.  The authenticator "ldapObject" allows for specifying the following fields related to the user search filter:

1. `user_search_prefix`
2. `user_search_supplemental_filter`
3. `user_search_filter`

See the JSONSchema definition for more details, but note that 1) and 2) are only used if 3) is not provided. 


## Restricting Access Via Host Filters

In case 3) defined in the Introduction section above, the tenant wishes to restrict access to a set of users 
specified by a host filter. In this case, we want the user search filter to include a `(host=<token>)` filter at the
end. For example, for the 3dem tenant, the search filter would look like this:

        (&(objectClass=person)(uid=?)(host=3dem.org))
        

The easiest way to do this is to create a custom IdP configuration in the Authenticator Postgres DB for the tenant with a single attrbute, `user_search_supplemental_filter` containing the host filter. For the 3dem example above, the JSON 
will look like:

      {"ldap": {"user_search_supplemental_filter": "(host=3dem.org)"}}


Here are the full steps to implementing this:

1. Exec into the authenticator-api container.
2. Start a python shell and execute:

```
from service.models import TenantConfig, db; import json
c = TenantConfig.query.filter_by(tenant_id='<your_tenant>')[0]
d = {'ldap': {'user_search_supplemental_filter': '(host=a2cps.org)'}}
s = json.dumps(d)
c.custom_idp_configuration = s
db.session.commit()
```

Note that if the tenant wishes to specify additional LDAP confiigurations (e.g., `default_page_limit_size`), then
those additional configurations should be specified within the definition of the `d` object above. 

## Other Kinds of IdPs in Authenticator via the Custom IdP Configuration

As mentioned previously, the Authenticator supports IdP definitions other than LDAP. The two main examples are:

1. Authentication with a third-party OAuth server, e.g., GitHub.
2. Authentication with KeyCloak to provide a federated identity solution.

In both cases, configuring the tenant with the such an IdP makes use of the custom_idp_configuration. Currently, a tenant can have either 0 or 1 custom_idp_configurations, which are JSON objects that conform to the JSONSchema defined in the authenticator repository here: 
    https://github.com/tapis-project/authenticator/blob/prod/service/resources/custom-idp-configschema.json
    
    
### GitHub OAuth ###
For GitHub auth, a `githubObject` definition should be supplied. The fields involved in the configuration are 
detailed in the JSONSchema, but in summary, a GitHub OAuth application must be created for Tapis authenticator 
and the corresponding client key and secret provided in the configuraiton. See ttps://docs.github.com/en/developers/apps/building-oauth-apps/creating-an-oauth-app for more information on creating GitHub OAuth clients.

The following steps can be used to store the `githubObject` configuration for a tenant:

1. Exec into the authenticator-api container.
2. Start a python shell and execute:

```
from service.models import TenantConfig, db
import json
c = TenantConfig.query.filter_by(tenant_id='github-demo')[0]
d = {'github': { 'client_id': '<you_client_key>', 'client_secret': '<your_client_secret>'}}
s = json.dumps(d)
c.custom_idp_configuration = s
db.session.commit()
```

